In [2]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import get_sdss_bands, get_sdss_colors

print('SAGA code version', SAGA.__version__)

SAGA code version 0.6.1


In [3]:
# import other modules

from collections import defaultdict
import numpy as np
from astropy.table import Table
from easyquery import Query

In [4]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)

saga_database = SAGA.Database('/home/yymao/Dropbox/Academia/Collaborations/SAGA', '/home/yymao/Documents/Research/SAGA')
saga_host_catalog = SAGA.HostCatalog(saga_database)
saga_object_catalog = SAGA.ObjectCatalog(saga_database)

In [5]:
# load paper1 base catalogs as a list

# specify columns to load
columns = 'OBJID HOST_NSAID RA DEC'.split()
columns.extend((c for c in C.COLUMNS_USED if c not in columns))

# specify cuts to apply
cuts = C.is_clean & C.is_galaxy & C.fibermag_r_cut & C.sat_rcut

# load base catalogs (will take a while)
objects_paper1 = saga_object_catalog.load('paper1', cuts=cuts, columns=columns, return_as='list', version='paper1')

In [6]:
# construct Table 1

host_info_cols = ['NSAID', 'RA', 'Dec', 'distance', 'M_r', 'M_K']

sat_Mr_compl = Query('M_r < -12.3')
counting_cols = [
    ('N_sat', C.is_sat & sat_Mr_compl),
    ('N_sat_below_compl', C.is_sat & ~sat_Mr_compl),
    ('N_total', C.faint_end_limit),
    ('N_gri_spec', C.faint_end_limit & C.gri_cut & C.has_spec),
    ('N_gri', C.faint_end_limit & C.gri_cut),
    ('N_ugri_spec', C.faint_end_limit & C.ugri_cut & C.has_spec),
    ('N_ugri', C.faint_end_limit & C.ugri_cut),
]

data = defaultdict(list)

for t in objects_paper1:
    host_nsaid = t['HOST_NSAID'][0]
    host_info = saga_host_catalog.load_single('nsa{}'.format(host_nsaid))
    
    t['M_r'] = t['r_mag'] - (5.0*(np.log10(host_info['distance'])+6.0)-5.0)
    
    data['SAGA_name'].append(host_info['SAGA_name'])

    for col in host_info_cols:
        data[col].append(host_info[col])
        
    for col, cut in counting_cols:
        data[col].append(cut.count(t))
        
Table(data)

SAGA_name,NSAID,RA,Dec,distance,M_r,M_K,N_sat,N_sat_below_compl,N_total,N_gri_spec,N_gri,N_ugri_spec,N_ugri
str11,int64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64
Bandamanna,126115,0.995575,20.7524,32.4741,-20.7677,-24.1403,1,0,2019,230,948,216,842
HarryPotter,129237,20.449,17.5922,38.6531,-20.0051,-23.5358,4,0,1526,53,832,47,742
Narnia,132339,39.5482,-1.31876,37.155,-21.0796,-23.4956,2,0,1976,778,849,688,750
StarTrek,33446,123.241,36.2546,37.7379,-21.2786,-23.4569,2,0,1719,716,842,662,765
MobyDick,85746,149.588,32.3699,25.1343,-20.2479,-23.0925,0,1,3635,604,1600,542,1351
Oz,137625,158.231,28.5118,24.4117,-20.7831,-23.0106,5,0,3801,142,1694,135,1487
Alice,140594,180.098,-1.10008,23.2487,-21.5041,-24.5004,2,0,5628,657,2681,618,2415
Sopranos,13927,180.676,1.9768,29.5168,-21.1343,-23.5937,0,0,3492,314,1888,299,1742
ScoobyDoo,161174,182.792,20.1757,36.3129,-20.6268,-23.0486,4,0,1471,353,758,331,675
